In [1]:
import numpy as np
import scipy.io as sio
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import csv
import os
import json
import math

from glob import glob
from collections import defaultdict
from scipy import signal
import matplotlib.pyplot as plt

In [2]:
jsonFileName = './inputJson/test.json'

f = open(jsonFileName)
inputJson = json.load(f)
dataPathBase = inputJson['dataPathBase']
dataType = inputJson['dataType']
f.close()

print(dataType)
if dataType == 'survey':
    fs = 1000 # 1 kHz
    nSubC = 30
    nRX = 3
    
    winLen = 1000
    thres = 60
    slideLen = 400

    # activities = ['bed']
    activities = ['bed', 'fall', 'pickup', 'run', 'sitdown', 'standup', 'walk']


survey


In [3]:
if dataType == "survey":
    dataPath = dataPathBase + "HAR_survey/"
    dataDict = {activity:[] for activity in activities}
    for activity in activities:
        dataDict[activity] = defaultdict(list)

elif dataType == "SHARP":
    dataPath = dataPathBase + "HAR_SHARP/"

for subFolder in os.walk(dataPath):
    if dataType == "survey":
        fileNameList = glob(subFolder[0] + "/input*.csv")
    print(len(fileNameList))
    
    for fileIndex, fileDirName in enumerate(fileNameList):
        fileName = fileDirName.split('/')[-1]
        fileType = fileName.split('_')[0]
        fileActv = fileName.split('_')[1]
        if fileActv not in activities:
            continue

        annotFileName = "annotation" + fileName[5:]
        annotFileDirName = subFolder[0] + annotFileName

        with open(fileDirName, mode='r') as file:
            csiData = list(csv.reader(file))
            csiDataNP = np.array(csiData).astype(np.float32)
            
        with open(annotFileDirName, mode='r') as annotFile:
            annotData = list(csv.reader(annotFile))
            annotDataNP = np.array(annotData)

        dataDict[fileActv]['CSI'].append(csiDataNP)
        dataDict[fileActv]['annot'].append(annotDataNP)

for activity in activities:
    print(activity, len(dataDict[activity]['annot']))

557
0
bed 79
fall 79
pickup 80
run 80
sitdown 80
standup 79
walk 80


In [4]:
for activity in activities:
    print('Checking annotation integrity of', activity, 'data')
    for dataInd in range(len(dataDict[activity]['annot'])):
        actDetected = 0
        for i in range(len(dataDict[activity]['annot'][dataInd])):
            # print(dataDict['run']['annot'][dataInd][i])
            if dataDict[activity]['annot'][dataInd][i] == activity:
                actDetected = 1
            elif actDetected == 1 and dataDict[activity]['annot'][dataInd][i] == 'NoActivity':
                actDetected = 2
            elif actDetected == 2 and dataDict[activity]['annot'][dataInd][i] == activity:
                print(activity, dataInd) # two activities in one file
                break

Checking annotation integrity of bed data
Checking annotation integrity of fall data
Checking annotation integrity of pickup data
Checking annotation integrity of run data
Checking annotation integrity of sitdown data
Checking annotation integrity of standup data
Checking annotation integrity of walk data


## Compute average length of activities

In [5]:
# activity = 'bed'
for activity in activities:
    actDurationSum = 0
    for dataInd in range(len(dataDict[activity]['annot'])):
        actDetected = 0
        minActStart = 0
        maxActEnd = 20000
        for i in range(len(dataDict[activity]['annot'][dataInd])):
            if actDetected == 0 and dataDict[activity]['annot'][dataInd][i] == activity:
                actDetected = 1
                actStart = i
            elif actDetected == 1 and dataDict[activity]['annot'][dataInd][i] == 'NoActivity':
                actEnd = i
                actDurationSum += (actEnd - actStart)
                print(dataInd, (actEnd - actStart), end=' ')
                # , math.floor((actEnd+actStart)/2)
                break
        if dataInd % 20 == 19 and dataInd != len(dataDict[activity]['annot'][dataInd])-1:
            print()
    print()
    actDurationAvg = actDurationSum / len(dataDict[activity]['annot'])
    print(activity, actDurationAvg)
    print()

0 2541 1 3861 2 3801 3 2821 4 4301 5 2981 6 3021 7 3191 8 2711 9 2981 10 4171 11 3491 12 3321 13 2541 14 4031 15 4341 16 3861 17 4811 18 3251 19 2811 
20 2711 21 2771 22 3421 23 3081 24 3451 25 3011 26 4641 27 3691 28 3631 29 3661 30 3661 31 3521 32 3901 33 3861 34 3521 35 4371 36 3191 37 3461 38 3251 39 3761 
40 3941 41 2851 42 3971 43 4131 44 3561 45 3791 46 2811 47 4271 48 3291 49 2951 50 1911 51 3491 52 4301 53 3051 54 4471 55 4131 56 3531 57 4271 58 3761 59 3971 
60 3461 61 4341 62 3251 63 4201 64 3221 65 3831 66 2951 67 3451 68 3021 69 3091 70 3051 71 3531 72 3971 73 3321 74 2911 75 4241 76 2881 77 4361 78 3831 
bed 3528.974683544304

0 1891 1 3191 2 2131 3 2441 4 2441 5 2981 6 2681 7 3181 8 1761 9 3081 10 2511 11 2551 12 2721 13 2511 14 3221 15 2851 16 2471 17 2921 18 2301 19 2511 
20 3191 21 2471 22 2781 23 2401 24 2581 25 2551 26 2411 27 1731 28 1861 29 2041 30 2911 31 2481 32 2371 33 2101 34 2551 35 2471 36 1631 37 2131 38 1561 39 2471 
40 2441 41 2541 42 2201 43 2751 44 2951

## Data visualization

In [ ]:
# for dataInd in range(len(dataDict[activity]['CSI'])):
activity = 'bed'
dataInd = 4

actDetected = 0
for i in range(len(dataDict[activity]['annot'][dataInd])):
    if actDetected == 0 and dataDict[activity]['annot'][dataInd][i] == activity:
        actDetected = 1
        actStart = i
    elif actDetected == 1 and dataDict[activity]['annot'][dataInd][i] == 'NoActivity':
        actEnd = i
        print(actStart, actEnd, end=' ')
        break

RXInd = 0
dataAmp = dataDict[activity]['CSI'][dataInd][:, 1:91]
fig = plt.figure(figsize = (18,10))
ax1 = plt.subplot(311)
plt.imshow(np.transpose(dataAmp[:, (RXInd*nSubC):(RXInd*nSubC+29)]), interpolation='nearest', aspect='auto')
ax1.set_title("Antenna1 Amp., entire duration")
plt.colorbar()

dataAmpNorm = dataAmp - np.mean(dataAmp, axis=0)
ax1 = plt.subplot(312)
plt.imshow(np.transpose(dataAmpNorm[:, (RXInd*nSubC):(RXInd*nSubC+29)]), interpolation='nearest', aspect='auto')
ax1.set_title("Antenna1 Amp., only duration with activity")
plt.colorbar()

ax1 = plt.subplot(313)
plt.imshow(np.transpose(dataAmpNorm[actStart:actEnd, (RXInd*nSubC):(RXInd*nSubC+29)]), interpolation='nearest', aspect='auto')
ax1.set_title("Antenna1 Amp., only duration with activity")
plt.colorbar()


# for i in range(1, len(dataAmpNorm[0])):
#     filtered_data[:,i] = moving_average(dataAmpNorm[:,i], 100)
